# Part II: Model Development

In this part, we develop three unique pipelines for predicting backorder. We use the smart sample from Part I to fit and evaluate these pipelines. 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
import joblib

from sklearn import preprocessing
from sklearn.model_selection import train_test_split


## Reload the smart sample here

In [2]:
# Reload your smart sampling from local file 
# ----------------------------------
X, y, train_undersamp = joblib.load('data/sample-data-v1.pkl')


## Normalize/standardize the data if required; otherwise ignore. You can perform this step inside the pipeline (if required). 

In [3]:
# Standardize data
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)
''' # Combine X_scaled and y
train_under_stand = pd.DataFrame()
train_under_stand = X_scaled
train_under_stand.reset_index(drop = True)
train_under_stand['went_on_backorder'] = y '''

print("Mean of the dataset features")
print(scaler.mean_)
print("Variance of data")
print(scaler.scale_)
print("-" * 35)

print("# Scaled data:")
print(X_scaled)

print("# Mean of scaled data")
print(X_scaled.mean(axis = 0))
print("# Variance of scaled data")
print(X_scaled.std(axis = 0))


Mean of the dataset features
[1.49783531e+02 7.01597165e+00 2.31098684e+01 1.49170901e+02
 3.75961981e+01 1.11286155e+02 3.39850410e+01 2.76166805e-03
 2.44987572e+00 7.41287858e-01 2.81487619e+00 1.85952315e-01
 5.98361410e-04 1.36242290e-01 9.74638682e-01 1.84111203e-04]
Variance of data
[2.05705652e+03 5.47785352e+00 4.83967322e+02 1.59776336e+03
 4.81613634e+02 1.40028856e+03 4.18233925e+02 5.24789599e-02
 4.14619611e+01 2.72113454e-01 6.30854082e+01 3.89068184e-01
 2.44541075e-02 3.43045666e-01 1.57219979e-01 1.35675092e-02]
-----------------------------------
# Scaled data:
[[-0.06989771 -0.91568196 -0.04775089 ... -0.39715497  0.16131104
  -0.01357001]
 [-0.06795318  0.90985061 -0.04775089 ...  2.51790883  0.16131104
  -0.01357001]
 [-0.06795318  0.17963758 -0.04775089 ... -0.39715497  0.16131104
  -0.01357001]
 ...
 [-0.07086997  0.17963758 -0.04775089 ... -0.39715497  0.16131104
  -0.01357001]
 [-0.06843931  0.90985061 -0.04775089 ... -0.39715497  0.16131104
  -0.01357001]
 [-

In [10]:
# Normalize data
train_undersamp_normalized = preprocessing.normalize(train_undersamp, axis = 0, norm = 'l2')
# Split into X and y
train_undersamp_norm = pd.DataFrame(train_undersamp_normalized)
train_undersamp_norm.columns=train_undersamp.columns
X_norm = train_undersamp_norm.iloc[:, :-1]
y_norm = train_undersamp_norm.went_on_backorder

print('# Scaled & normalized values:')
print(train_undersamp_normalized)

print("# All have unit norm")
print(np.linalg.norm(train_undersamp_normalized, axis = 0))


# Scaled & normalized values:
[[1.97363517e-05 1.52438006e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 0.00000000e+00]
 [3.28939195e-05 9.14628036e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 0.00000000e+00]
 [3.28939195e-05 6.09752024e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 0.00000000e+00]
 ...
 [1.31575678e-05 6.09752024e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 9.59456104e-03]
 [2.96045275e-05 9.14628036e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 9.59456104e-03]
 [6.57878390e-06 6.85971027e-03 0.00000000e+00 ... 6.87208133e-03
  0.00000000e+00 9.59456104e-03]]
# All have unit norm
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [22]:
train_undersamp_norm.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
national_inv,21726.0,0.000493,0.006767,-0.026743,0.000003,0.000016,0.000076,0.699387
lead_time,21726.0,0.005348,0.004175,0.000000,0.001524,0.006098,0.006098,0.039634
in_transit_qty,21726.0,0.000324,0.006777,0.000000,0.000000,0.000000,0.000000,0.696251
forecast_3_month,21726.0,0.000631,0.006755,0.000000,0.000000,0.000017,0.000106,0.507334
sales_1_month,21726.0,0.000528,0.006764,0.000000,0.000000,0.000014,0.000098,0.622923
sales_3_month,21726.0,0.000537,0.006763,0.000000,0.000000,0.000019,0.000101,0.666536
min_bank,21726.0,0.000549,0.006762,0.000000,0.000000,0.000000,0.000065,0.556251
potential_issue,21726.0,0.000357,0.006775,0.000000,0.000000,0.000000,0.000000,0.129099
pieces_past_due,21726.0,0.000400,0.006773,0.000000,0.000000,0.000000,0.000000,0.506367
perf_6_month_avg,21726.0,0.006369,0.002338,0.000000,0.005585,0.007045,0.008248,0.008592


## Split the data into Train/Test

In [11]:
# Split original dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
# Split normalized dataset into training and testing sets
X_train_norm, X_test_norm, y_train_norm, y_test_norm = train_test_split(X, y, test_size = 0.2)


## Developing Pipeline

In this section, we design an operationalized machine learning pipeline, which includes:

* Anomaly detection
* Dimensionality Reduction
* Train a classification model


We are free to use any of the models that we learned in the past or we can use new models. Here is a pool of methods: 

### Pool of Anomaly Detection Methods (Discussed in M4)
1. IsolationForest
2. EllipticEnvelope
3. LocalOutlierFactor
4. OneClassSVM
5. SGDOneClassSVM

### Pool of Feature Selection Methods (Discussed in M3)

1. VarianceThreshold
1. SelectKBest with any scoring method (e.g, chi, f_classif, mutual_info_classif)
1. SelectKPercentile
3. SelectFpr, SelectFdr, or  SelectFwe
1. GenericUnivariateSelect
2. PCA
3. Factor Analysis
4. Variance Threshold
5. RFE
7. SelectFromModel


### Classification Methods (Discussed in M1-M2
1. Decision Tree
2. Random Forest
3. Logistic Regression
4. Naive Bayes
5. Linear SVC
6. SVC with kernels
7. KNeighborsClassifier
8. GradientBoostingClassifier
9. XGBClassifier
10. LGBM Classifier



It is difficult to fit an anomaly detection method in the sklearn pipeline without writing custom codes. For simplicity, we avoid fitting an anomaly detection method within a pipeline. So we can create the workflow in two steps. 
* Step I: fit an outlier with the training set
* Step II: define a pipeline using a feature selection and a classification method. Then cross-validate this pipeline using the training data without outliers. 
* Note: if your smart sample is somewhat imbalanced, you might want to change the scoring method in GridSearchCV (see the [doc](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)).


Once we fit the pipeline with gridsearch, we identify the best model and give an unbiased evaluation using the test set that we created in Part II. For unbiased evaluation we report confusion matrix, precision, recall, f1-score, accuracy, and other measures if you like. 

**Optional: Those who are interested in writing custom codes for adding an outlier detection method into the sklearn pipeline, please follow this discussion [thread](https://stackoverflow.com/questions/52346725/can-i-add-outlier-detection-and-removal-to-scikit-learn-pipeline).**


**Note:** <span style='background:yellow'>We will be using Grid Search to find the optimal parameters of the pipelines.</span>

You can add more notebook cells or import any Python modules as needed.

In [12]:
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix 
from numpy import where


### Your 1st pipeline 
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation
  
Add cells as needed. 

In [13]:
# Add anomaly detection code  (Question #E201)
# ----------------------------------
# Construct local outlier factor
lof = LocalOutlierFactor(n_neighbors = 20, contamination = .03)

# Fit model 
# Get labels from classifier to cull outliers
outliers = lof.fit_predict(X_train) == -1

X_lof = X_train[~outliers]
y_lof = y_train[~outliers]


In [14]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E202)
# ----------------------------------
pca = PCA(n_components = 5, random_state = 17)
lr = LogisticRegression()

# Define pipeline
pipe = Pipeline([
    ('PCA', pca),
    ('LR_model', lr)
])

# Define parameter grid
param_grid = {'PCA__n_components': [5, 20, 30, 35, 60], 
              'LR_model__C': [0.001, 0.1, 1.0, 10, 100], 
              'LR_model__max_iter': [1000]
              }

# Use Grid Search to train Pipeline
CV_log_reg = GridSearchCV(pipe, param_grid, n_jobs = 2, cv = 10)
CV_log_reg.fit(X_lof, y_lof)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [0.70245675        nan        nan        nan        nan 0.70251606
        nan        nan        nan        nan 0.70251606        nan
        nan        nan        nan 0.70251606        nan        nan
        nan        nan 0.70251606        nan        nan        nan
        nan]
  category=UserWarning


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('PCA',
                                        PCA(n_components=5, random_state=17)),
                                       ('LR_model', LogisticRegression())]),
             n_jobs=2,
             param_grid={'LR_model__C': [0.001, 0.1, 1.0, 10, 100],
                         'LR_model__max_iter': [1000],
                         'PCA__n_components': [5, 20, 30, 35, 60]})

In [18]:
# Given an unbiased evaluation  (Question #E203)
# ----------------------------------
# Show parameters of trained models and their rank
pd.set_option("max_colwidth", 80)
CV_log_reg_df = pd.DataFrame(CV_log_reg.cv_results_)
CV_log_reg_df[['params','rank_test_score']]
#Show parameters of best model
best_params = CV_log_reg.best_params_
print('Best parameter:\n',best_params)

# Evaluate best model using test data
predicted_y = CV_log_reg.predict(X_test)

# Display confusion matrix
print('\nConfusion Matrix:\n',pd.DataFrame(confusion_matrix(y_test, predicted_y)))

# Create classification report
print('\nClassification Report:\n',classification_report(y_test, predicted_y))


Best parameter:
 {'LR_model__C': 0.1, 'LR_model__max_iter': 1000, 'PCA__n_components': 5}

Confusion Matrix:
      0     1
0  856  1294
1   87  2109

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.40      0.55      2150
           1       0.62      0.96      0.75      2196

    accuracy                           0.68      4346
   macro avg       0.76      0.68      0.65      4346
weighted avg       0.76      0.68      0.65      4346



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 2nd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [19]:
# Add anomaly detection code  (Question #E205)
# ----------------------------------
# Construct envelope
env = EllipticEnvelope()
# Fit data to envelope
envelope = env.fit(X_train_norm, y_train_norm)

# Get labels from classifier to cull outliers
outliers = envelope.predict(X_train) == -1

# Re-slice X,y into a cleaned dataset with outliers excluded
X_env = X_train[~outliers]
y_env = y_train[~outliers]


In [20]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E206)
# ----------------------------------
# Construct random forrest classifier
rfc = RandomForestClassifier(random_state = 17)
kb = SelectKBest(score_func = chi2, k = 3)
# TO DO: Look for something other than chi2

pipe = Pipeline([
    ('kbest', kb), 
    ('rfc_model', rfc)
])

n_components = [4, 8, 16]
K = [3, 5, 10, 15, 20]

param_grid = [
    {
        'kbest__k': K,
        'rfc_model__max_features' : ['auto', 'sqrt', 'log2'],
        'rfc_model__max_depth': [None, 15, 10, 5, 1],
        'rfc_model__n_estimators':[25, 50, 100]
    },
]

# Use Grid Search to train Pipeline
CV_rfc = GridSearchCV(pipe, param_grid = param_grid, n_jobs = 2, cv = 10)
CV_rfc.fit(X_env, y_env)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan n

ValueError: Input X must be non-negative.

In [ ]:
# Given an unbiased evaluation  (Question #E207)
# ----------------------------------



#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## <span style="background: yellow;">Commit your code!</span> 

### Your 3rd pipeline
  * Anomaly detection
  * Dimensionality reduction
  * Model training/validation

In [ ]:
# Add anomaly detection code  (Question #E209)
# ----------------------------------
# Construct IsolationForest 
iso_forest = IsolationForest(n_estimators = 250,bootstrap = True).fit(X_train, y_train)

# Get labels from classifier to cull outliers
iso_outliers = iso_forest.predict(X_train) == -1

X_iso = X_train[~iso_outliers]
y_iso = y_train[~iso_outliers]


In [ ]:
# Add codes for feature selection and classification pipeline with grid search  (Question #E210)
# ----------------------------------


In [ ]:
# Given an unbiased evaluation  (Question #E211)
# ----------------------------------

#### <center>Record the optimal hyperparameters and performance resulting from this pipeline.</center>

## Compare these three pipelines and discuss your findings

## <span style="background: yellow;">Commit your code!</span> 

### Pickle the required pipeline/models for Part III.

You should have made a few commits so far of this project.  
**Definitely make a commit of the notebook now!**  
Comment should be: `Final Project, Checkpoint - Pipelines done`


# Save your notebook!
## Then `File > Close and Halt`